In [5]:
import mne
import numpy as np
import matplotlib as mp
import pandas as pd
from mne.channels import read_layout
from mne.preprocessing import ICA
import matplotlib.pyplot as plt




from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws
from mne.decoding import CSP

In [6]:
data=np.load('dataarray.npy')
cols=['FCz', 'FC2', 'FC4', 'FC6', 'Cz', 'C2', 'C4', 'C6', 'Cpz', 'Cp2', 'Cp4', 'Cp6','pz','stim']
#df=pd.DataFrame(data=data.T,,columns=cols)
print(data.shape)
datastack=np.reshape(data,(10*14,18000))

(10, 14, 18000)


In [14]:
[ntrail,nchann,nsamp]=data.shape
sampling_freq = 128  # in Hertz
ch_names = ['FCz', 'FC2', 'FC4', 'FC6', 'Cz', 'C2', 'C4', 'C6', 'CPz', 'CP2', 'CP4', 'CP6','A2','STI 001']
ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg','eeg','stim']

info = mne.create_info(ch_names=ch_names, sfreq=sampling_freq, ch_types=ch_types)
raw= mne.io.RawArray(data[1,:,:], info)
raw = concatenate_raws([mne.io.RawArray(data[f,:,:], info) for f in range(0,ntrail)])
#montage = make_standard_montage('standard_1020')
raw.set_montage('standard_1020')


Creating RawArray with float64 data, n_channels=14, n_times=18000
    Range : 0 ... 17999 =      0.000 ...   140.617 secs
Ready.
Creating RawArray with float64 data, n_channels=14, n_times=18000
    Range : 0 ... 17999 =      0.000 ...   140.617 secs
Ready.
Creating RawArray with float64 data, n_channels=14, n_times=18000
    Range : 0 ... 17999 =      0.000 ...   140.617 secs
Ready.
Creating RawArray with float64 data, n_channels=14, n_times=18000
    Range : 0 ... 17999 =      0.000 ...   140.617 secs
Ready.
Creating RawArray with float64 data, n_channels=14, n_times=18000
    Range : 0 ... 17999 =      0.000 ...   140.617 secs
Ready.
Creating RawArray with float64 data, n_channels=14, n_times=18000
    Range : 0 ... 17999 =      0.000 ...   140.617 secs
Ready.
Creating RawArray with float64 data, n_channels=14, n_times=18000
    Range : 0 ... 17999 =      0.000 ...   140.617 secs
Ready.
Creating RawArray with float64 data, n_channels=14, n_times=18000
    Range : 0 ... 17999 =      

<RawArray | 14 x 180000 (1406.2 s), ~19.3 MB, data loaded>

In [17]:
events = mne.find_events(raw, 'STI 001')
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')
tmin, tmax = -3, 4  # define epochs around events (in s)
event_ids = dict(MI=2)  # map event IDs to tasks

raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')
epochs = Epochs(raw, events, event_ids, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)
epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
labels = epochs.events[:, -1] - 2
print(labels)

284 events found
Event IDs: [1 2 3]
Filtering raw data in 10 contiguous segments
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 213 samples (1.664 sec)

Not setting metadata
Not setting metadata
95 matching events found
No baseline correction applied
0 projection items activated
Loading data for 95 events and 897 original time points ...
2 bad epochs dropped
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
